In [99]:
from sympy import symbols
from sympy.logic.boolalg import to_cnf
from sympy import And, Or, Not
import numpy as np
import pyapproxmc

In [227]:
def read_tiles_from_file(filename):
    tiles_file = open(filename, "r")
    glue_lines = tiles_file.readlines()
    for i in range(len(glue_lines)):
        glue_lines[i] = glue_lines[i].split()
        # splits into an array of multiple tiles per position, which we can access afterwards
    return glue_lines

In [228]:
def variable_creator(glue_lines, n):
    symbols_matrix = []
    for position_ind in range(len(glue_lines)): 
        symbols_row = []
        row = int(position_ind / n)
        col = int(position_ind % n)
        for tile_type_pp in range(len(glue_lines[position_ind])):
            symbols_row += [symbols(f'T_{row}_{col}_{tile_type_pp}')]
        symbols_matrix += [symbols_row]

    return symbols_matrix

In [229]:
tile_def = 'rand_t4pp'
glues = read_tiles_from_file(f'./data/{tile_def}.txt')
total_positions = len(glues)
symbols_lines = variable_creator(glues, 5)

In [230]:
def occupancy_clause_creator(symbols_lines):
    position_occupancy_clauses = True
        
    for pos in range(total_positions):
        pos_symbols = symbols_lines[pos]
        presence_clause = False
        for i in range(len(pos_symbols)):
            presence_clause = presence_clause | pos_symbols[i]
            for j in range(i+1, len(pos_symbols)):
                multiple_occupancy_clause = ~pos_symbols[i] | ~pos_symbols[j]
                position_occupancy_clauses = position_occupancy_clauses & (multiple_occupancy_clause)
        position_occupancy_clauses = position_occupancy_clauses & presence_clause
    print(f'Initial Formula: {position_occupancy_clauses}')

    return position_occupancy_clauses

    # add adjacency formulas after this by accessing the glue and comparing them...
    # use the C code to help guide how the internal bonds are checked...

In [231]:
occupancy_clauses = occupancy_clause_creator(symbols_lines)

Initial Formula: (~T_0_0_0 | ~T_0_0_1) & (~T_0_0_0 | ~T_0_0_2) & (~T_0_0_0 | ~T_0_0_3) & (~T_0_0_1 | ~T_0_0_2) & (~T_0_0_1 | ~T_0_0_3) & (~T_0_0_2 | ~T_0_0_3) & (~T_0_1_0 | ~T_0_1_1) & (~T_0_1_0 | ~T_0_1_2) & (~T_0_1_0 | ~T_0_1_3) & (~T_0_1_1 | ~T_0_1_2) & (~T_0_1_1 | ~T_0_1_3) & (~T_0_1_2 | ~T_0_1_3) & (~T_0_2_0 | ~T_0_2_1) & (~T_0_2_0 | ~T_0_2_2) & (~T_0_2_0 | ~T_0_2_3) & (~T_0_2_1 | ~T_0_2_2) & (~T_0_2_1 | ~T_0_2_3) & (~T_0_2_2 | ~T_0_2_3) & (~T_0_3_0 | ~T_0_3_1) & (~T_0_3_0 | ~T_0_3_2) & (~T_0_3_0 | ~T_0_3_3) & (~T_0_3_1 | ~T_0_3_2) & (~T_0_3_1 | ~T_0_3_3) & (~T_0_3_2 | ~T_0_3_3) & (~T_0_4_0 | ~T_0_4_1) & (~T_0_4_0 | ~T_0_4_2) & (~T_0_4_0 | ~T_0_4_3) & (~T_0_4_1 | ~T_0_4_2) & (~T_0_4_1 | ~T_0_4_3) & (~T_0_4_2 | ~T_0_4_3) & (~T_1_0_0 | ~T_1_0_1) & (~T_1_0_0 | ~T_1_0_2) & (~T_1_0_0 | ~T_1_0_3) & (~T_1_0_1 | ~T_1_0_2) & (~T_1_0_1 | ~T_1_0_3) & (~T_1_0_2 | ~T_1_0_3) & (~T_1_1_0 | ~T_1_1_1) & (~T_1_1_0 | ~T_1_1_2) & (~T_1_1_0 | ~T_1_1_3) & (~T_1_1_1 | ~T_1_1_2) & (~T_1_1_1 | ~T_1_1_3) &

In [232]:
# input a negative value for mismatch_loc if you are looking for the tiling with no mismatches
def adjacency_clause_creator(symbols_lines, glue_lines, n, mismatch_loc):
    apply_mismatch = False
    if (mismatch_loc >= 0):
        apply_mismatch = True
    mismatch_row = int(mismatch_loc / (2 * n - 1))
    loc = mismatch_loc % (2 * n - 1)
    mismatch_col = loc
    if (loc > n - 2):
        mismatch_col = loc - n + 1

    adjacency_clauses = True
    for pos in range(total_positions):
        row = int(pos / n)
        col = int(pos % n) 
        for tile_type_pp in range(len(symbols_lines[pos])):
            nesw = glue_lines[pos][tile_type_pp]
            curr_south = nesw[2]
            curr_east = nesw[1]

            if row < n - 1:
                for bot_tile_type_pp in range(len(symbols_lines[pos + n])):
                    bot_nesw = glue_lines[pos + n][bot_tile_type_pp]
                    bot_north = bot_nesw[0]
                    if (apply_mismatch and row == mismatch_row and col == mismatch_col and loc > n - 2):
                        # enforce a mismatch in this position (south bond of mismatch row and mismatch col)
                        if (curr_south == bot_north):
                            adjacency_clauses = adjacency_clauses & (~symbols_lines[pos][tile_type_pp] | ~symbols_lines[pos + n][bot_tile_type_pp])
                    else:
                        if (curr_south != bot_north):
                            adjacency_clauses = adjacency_clauses & (~symbols_lines[pos][tile_type_pp] | ~symbols_lines[pos + n][bot_tile_type_pp])
            if col < n - 1:
                for right_tile_type_pp in range(len(symbols_lines[pos + 1])):
                    right_nesw = glue_lines[pos + 1][right_tile_type_pp]
                    right_west = right_nesw[3]
                    if (apply_mismatch and row == mismatch_row and col == mismatch_col and loc < n - 1):
                        # enforce a mismatch in this position (right bond of mismatch row and mismatch col)
                        if (curr_east == right_west):
                            adjacency_clauses = adjacency_clauses & (~symbols_lines[pos][tile_type_pp] | ~symbols_lines[pos + 1][right_tile_type_pp])
                    else:
                        if (curr_east != right_west):
                            adjacency_clauses = adjacency_clauses & (~symbols_lines[pos][tile_type_pp] | ~symbols_lines[pos + 1][right_tile_type_pp])
    return adjacency_clauses

In [233]:
adjacency_clauses = adjacency_clause_creator(symbols_lines, glues, 5, -1)

In [234]:
valid_tiling_formula = occupancy_clauses & adjacency_clauses

In [235]:
mismatch_tiling_formulas = [occupancy_clauses & adjacency_clause_creator(symbols_lines, glues, 5, i) for i in range(40)]

In [236]:
def sympy_to_pyapproxmc(sympy_cnf, symbols_list):
    # Create a mapping from SymPy symbols to integers
    symbol_to_int = {str(symbol): i + 1 for i, symbol in enumerate(symbols_list)}
    clauses = []

    # Helper function to convert SymPy literal to pyapproxmc literal
    def literal_to_pyapproxmc(literal):
        if literal.is_Symbol:
            return symbol_to_int[str(literal)]
        elif literal.func == Not:
            return -symbol_to_int[str(literal.args[0])]
        else:
            raise ValueError("Unexpected literal type")

    # Process the CNF clauses
    if sympy_cnf.func == And:
        cnf_clauses = sympy_cnf.args
    else:
        cnf_clauses = [sympy_cnf]

    for clause in cnf_clauses:
        if clause.func == Or:
            pyapproxmc_clause = [literal_to_pyapproxmc(lit) for lit in clause.args]
        else:
            pyapproxmc_clause = [literal_to_pyapproxmc(clause)]
        clauses.append(pyapproxmc_clause)

    return clauses

In [237]:
def approximate_solutions(tiling_formula, symbols_lines, i):
    cnf_formula = to_cnf(tiling_formula)
    pyapproxmc_clauses = sympy_to_pyapproxmc(cnf_formula, np.ravel(symbols_lines))
    # Count solutions using pyapproxmc
    counter = pyapproxmc.Counter()
    counter.add_clauses(pyapproxmc_clauses)
    num_solutions = counter.count()
    # print(f'Approximate number of solutions: {num_solutions}')
    print("%s: Approximate count is: %d*2**%d" % (str(i), num_solutions[0], num_solutions[1]))
    return num_solutions

In [238]:
approximate_solutions(valid_tiling_formula, symbols_lines, 'Valid tiling')

Valid tiling: Approximate count is: 0*2**0


(0, 0)

In [239]:
one_mismatch_tilings_out = [approximate_solutions(mismatch_tiling_formulas[i], symbols_lines, i) for i in range(len(mismatch_tiling_formulas))]
one_mismatch_tilings = [out[0] * 2 ** out[1]for out in one_mismatch_tilings_out]
print(np.sum(one_mismatch_tilings))

0: Approximate count is: 0*2**0
1: Approximate count is: 0*2**0
2: Approximate count is: 0*2**0
3: Approximate count is: 0*2**0
4: Approximate count is: 0*2**0
5: Approximate count is: 0*2**0
6: Approximate count is: 0*2**0
7: Approximate count is: 0*2**0
8: Approximate count is: 0*2**0
9: Approximate count is: 0*2**0
10: Approximate count is: 0*2**0
11: Approximate count is: 0*2**0
12: Approximate count is: 0*2**0
13: Approximate count is: 0*2**0
14: Approximate count is: 0*2**0
15: Approximate count is: 0*2**0
16: Approximate count is: 0*2**0
17: Approximate count is: 0*2**0
18: Approximate count is: 0*2**0
19: Approximate count is: 0*2**0
20: Approximate count is: 0*2**0
21: Approximate count is: 0*2**0
22: Approximate count is: 0*2**0
23: Approximate count is: 0*2**0
24: Approximate count is: 0*2**0
25: Approximate count is: 0*2**0
26: Approximate count is: 0*2**0
27: Approximate count is: 0*2**0
28: Approximate count is: 0*2**0
29: Approximate count is: 0*2**0
30: Approximate coun

In [161]:
def sympy_to_dimacs(sympy_cnf, symbols_list):
    # Create a mapping from SymPy symbols to integers
    symbol_to_int = {str(symbol): i + 1 for i, symbol in enumerate(symbols_list)}
    dimacs_clauses = []

    # Helper function to convert SymPy literal to DIMACS literal
    def literal_to_dimacs(literal):
        if literal.is_Symbol:
            return symbol_to_int[str(literal)]
        elif literal.func == Not:
            return -symbol_to_int[str(literal.args[0])]
        else:
            raise ValueError("Unexpected literal type")

    # Process the CNF clauses
    if sympy_cnf.func == And:
        clauses = sympy_cnf.args
    else:
        clauses = [sympy_cnf]

    for clause in clauses:
        if clause.func == Or:
            dimacs_clause = [literal_to_dimacs(lit) for lit in clause.args]
        else:
            dimacs_clause = [literal_to_dimacs(clause)]
        dimacs_clauses.append(dimacs_clause)

    # Generate DIMACS string
    num_vars = len(symbols_list)
    num_clauses = len(dimacs_clauses)
    dimacs_str = f'p cnf {num_vars} {num_clauses}\n'
    for clause in dimacs_clauses:
        dimacs_str += ' '.join(map(str, clause)) + ' 0\n'

    return dimacs_str

In [162]:
# # Convert SymPy CNF to PySAT CNF/
# dimacs_cnf = sympy_to_dimacs(cnf_valid_tiling_formula, np.ravel(symbols_lines))
# print(dimacs_cnf)

In [163]:
# Save DIMACS to a temporary file
# with open('single-yes-t2pp.cnf', 'w') as f:
#     f.write(dimacs_cnf)